In [71]:
import datetime
from datetime import datetime, timedelta
import calendar
import requests
import json
import urllib3
import configparser
config = configparser.ConfigParser()
config.read('credentials.ini')
MDM_add = config['API']['MDM_add']
MDM_user = config['API']['MDM_username']
MDM_password = config['API']['MDM_password']
HES_add = config['API']['HES_add']
HES_user = config['API']['HES_username']
HES_password = config['API']['HES_password']

In [72]:
def get_devices(add,user,password):
    url= f"{add}/api/1/devices/"
    try:
        r = requests.get(url1, auth=(user, password), verify=False)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return []
    
    device = r.json()
    
    del_key = ['communicationId', 'typeId', 'typeName', 'templateId', 'templateName', 'managementState', 'description', 'manufacturer', 'model', 'parentId', 'location', 'storeData', 'groupId']
    for item in device:
        for k in del_key:
            item.pop(k, None)
    return device

In [93]:
print(f"{'='*60}\n********Data Will Be Extracted For Following********\n{'='*60}")
print(f"Data Source   : {data_source}")
print(f"Profile       : {tprof}")
print(f"Profile Value : {prof}")
print(f"Start Time    : {start_time_cur}")
print(f"End Time      : {to_time_cur}")
print(f"{'='*150}\n")

********Data Will Be Extracted For Following********
Data Source   : MDM


NameError: name 'tprof' is not defined

In [73]:
print("***** Where do you want to get data from HES or MDM*****")

***** Where do you want to get data from HES or MDM*****


In [86]:
print("***** Where do you want to get data from HES or MDM*****")
while True:
    data_source = input("Enter the data source (HES or MDM): ").strip()

    if data_source == "HES":
        add = HES_add
        user = HES_user
        password = HES_password
        break
    elif data_source == "MDM":
        add = MDM_add
        user = MDM_user
        password = MDM_password
        break
    else:
        print("Invalid choice. Please enter either 'HES' or 'MDM'.")
print(f"********** Data Source is {data_source} **********")

***** Where do you want to get data from HES or MDM*****


Enter the data source (HES or MDM):  MDM


***** Data Source is MDM *****


In [82]:
def billing_time_data(month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    # Use current year and month if not provided
    now = datetime.now()
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    
    month = month.capitalize()
    cur_start_date = datetime(year, list(calendar.month_name).index(month), 
                              calendar.monthrange(year, list(calendar.month_name).index(month))[1],
                              start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")        
    return start_time_cur, end_time_cur


def daily_time_data(day: int = None, month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    now = datetime.now()
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    month = month.capitalize()
    if day is None:
        day = now.day
        month = now.strftime('%B')
        year = now.year  
    cur_start_date = datetime(year, list(calendar.month_name).index(month), day, start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_time_cur, end_time_cur


In [83]:

while True:
    profile = input("Enter the profile you want to extract data for (Billing or Daily): ").strip().lower()
    if profile == "billing":
        prof = "1-0:98.1.0*255"
        while True:
            try:
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                start_time_cur, to_time_cur = billing_time_data(month, year)
                break
            except (ValueError, IndexError):
                print("Invalid date. Please try again.")
        break
    elif profile == "daily":
        prof = "1-0:99.2.0*255"
        while True:
            try:
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                day_input = input("Enter the day (e.g., 31) or press Enter to use today's date: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                day = int(day_input) if day_input else None
                start_time_cur, to_time_cur = daily_time_data(day, month, year)
                break
            except (ValueError, IndexError):
                print("Invalid Date. Please try again")
        break
            
    else:
        print("Invalid choice. Please enter either 'billing' or 'daily'.")
        
end_date_part = start_time_cur.split("T")[0]


Enter the profile you want to extract data for (Billing or Daily):  Billing
Enter the month (e.g., January) or press Enter to use the current month:  
Enter the year (e.g., 2024) or press Enter to use the current year:  


In [87]:
print(data_source)
print(prof)
print(start_time_cur)
print(to_time_cur)
print(end_date_part)

MDM
1-0:98.1.0*255
2024-09-30T18:29:00Z
2024-09-30T18:31:00Z
2024-09-30


In [91]:
#print(f"Data Will Be Extracted For Following:\n{'='*38}")
print(f"Data Will Be Extracted For Following:\n{'='*38}")

print(f"Data Source   : {data_source}")
print(f"Profile       : {profile.capitalize()} Profile")
print(f"Start Time    : {start_time_cur}")
print(f"End Time      : {to_time_cur}")

Data Will Be Extracted For Following:
Data Source   : MDM
Profile       : Billing Profile
Start Time    : 2024-09-30T18:29:00Z
End Time      : 2024-09-30T18:31:00Z
